<a href="https://colab.research.google.com/github/Chris1Areiza/gcsproof/blob/main/Modelo_Cita_de_entrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
import cx_Oracle
import pandas as pd

# Conexión a la base de datos
conn = cx_Oracle.connect('usuario/contraseña@nombre_de_servidor:puerto/nombre_de_BD')

# Consulta SQL para leer la tabla
query = 'SELECT * FROM nombre_de_tabla'

# Leer los datos en un DataFrame de pandas
df = pd.read_sql(query, con=conn)

# Cerrar la conexión
conn.close()

# Imprimir el DataFrame
print(df.head())
'''

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [2]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Leer el archivo con los datos
df = pd.read_excel('drive/MyDrive/data_sample.xlsx', index_col = False)

# Seleccionar las columnas necesarias
df = df[["Proveedor", "Categoría", "Subcategoría", "Sublínea", "Días retraso"]]

# Calcular el rango intercuartil
Q1 = df["Días retraso"].quantile(0.25)
Q3 = df["Días retraso"].quantile(0.75)
IQR = Q3 - Q1

# Definir límites inferior y superior
lim_inf = Q1 - 1.5 * IQR
lim_sup = Q3 + 1.5 * IQR

# Reemplazar valores atípicos inferiores por percentil 5 y superiores por percentil 95
df.loc[df["Días retraso"] < lim_inf, "Días retraso"] = df["Días retraso"].quantile(0.05)
df.loc[df["Días retraso"] > lim_sup, "Días retraso"] = df["Días retraso"].quantile(0.95)

# Agrupar los datos por sublínea y calcular la media y desviación estándar
df_grouped = df.groupby(["Proveedor", "Categoría", "Subcategoría", "Sublínea"]).agg({"Días retraso": [np.mean, np.std]}).reset_index()

# Cambiar el nombre de las columnas
df_grouped.columns = ["Proveedor", "Categoría", "Subcategoría", "Sublínea", "Media", "Desviación"]

# Calcular la inversa de la normal para una probabilidad dada
probabilidad = input()
df_grouped["Lead Time"] = norm.ppf(0.8, df_grouped["Media"], df_grouped["Desviación"]).round(decimals=0)

# Reemplazar los valores NaN en la columna Inversa de la normal por el valor de la columna Media
df_grouped["Lead Time"] = df_grouped["Lead Time"].fillna(df_grouped["Media"])

# Mostrar el resultado
print(df_grouped)


0.8
   Proveedor Categoría Subcategoría Sublínea      Media  Desviación  Lead Time
0   ARTDECON        DO           AD      CHL   3.416667    3.315483        6.0
1   ARTDECON        DO           DB       CB   0.545455    1.213560        2.0
2   ARTDECON        DO           DB      CDB   5.090909    5.459017       10.0
3   ARTDECON        DO           DB       NO  13.863636    5.548878       19.0
4   ARTDECON        DO           DB       TO  11.928571    5.623552       17.0
5   ARTDECON        DO           JU      CSB   1.857143    2.340126        4.0
6   ARTDECON        DO           JU       DU   7.000000         NaN        7.0
7   ARTDECON        SO           CM       BI  14.875000    6.664136       20.0
8   ARTDECON        SO           CM    C4PTO  11.333333    5.859465       16.0
9   ARTDECON        SO           CM    C6PTO  13.875000    7.900045       21.0
10  ARTDECON        SO           CT      MRE  15.142857    3.625308       18.0
11  ARTDECON        SO           CT       MS  14

/usr/local/lib/python3.9/dist-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/usr/local/lib/python3.9/dist-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [4]:
! git status


fatal: not a git repository (or any of the parent directories): .git
